In [10]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm  # 진행 상황 표시용 (pip install tqdm)
from sentence_transformers import SentenceTransformer

def preprocess_csv_with_embedding_gpu(
    file_path,
    output_path,
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    min_length=5,
    similarity_threshold=0.9
):
    # 1. CSV 로드
    print("CSV 파일 로드 중...")
    df = pd.read_csv(file_path)
    
    # 2. 기본 전처리: 숫자로 시작하는 부분, 큰따옴표, 모든 숫자 제거
    print("텍스트 전처리 중... (숫자, 큰따옴표 제거)")
    df["generated_dialogue"] = df["generated_dialogue"].str.replace(
        r'^\d+\.?\s*', '', regex=True
    )
    df["generated_dialogue"] = df["generated_dialogue"].str.replace(
        r'"', '', regex=True
    )
    df["generated_dialogue"] = df["generated_dialogue"].str.replace(
        r'[0-9]+', '', regex=True
    )
    df.rename(columns={"generated_dialogue": "text"}, inplace=True)
    
    # 3. 너무 짧은 문장 제거
    df = df[df["text"].str.len() >= min_length].reset_index(drop=True)
    print(f"전처리 후 남은 문장 수: {len(df)}")
    
    # 4. GPU 사용 여부 확인
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("사용 중인 디바이스:", device)
    
    # 5. 모델 로드 (GPU/CPU 자동 설정)
    print("모델 로드 중...")
    model = SentenceTransformer(model_name, device=device)
    print("모델이 로드되었습니다. 모델 디바이스:", model._target_device)
    
    # 6. 임베딩 생성 (GPU에서 생성)
    print("임베딩 생성 중...")
    texts = df["text"].tolist()
    embeddings = model.encode(texts, convert_to_tensor=True)  # Tensor, device=device
    
    # 7. GPU에서 코사인 유사도 계산 (벡터화 연산)
    print("GPU에서 코사인 유사도 계산 중...")
    # 정규화
    embeddings_norm = torch.nn.functional.normalize(embeddings, p=2, dim=1)
    # 전체 유사도 행렬 계산: (n, n)
    similarity_matrix = torch.matmul(embeddings_norm, embeddings_norm.T)
    # similarity_matrix는 GPU에 있으므로, 이후 반복문 처리를 위해 CPU로 옮깁니다.
    similarity_matrix = similarity_matrix.cpu().numpy()
    
    # 8. 코사인 유사도 기반 중복 제거 (벡터화된 유사도 사용)
    print("유사 문장 제거 중...")
    to_remove = set()
    n = len(df)
    for i in tqdm(range(n), desc="중복 제거 진행 중"):
        if i in to_remove:
            continue
        # i번째 행의 i+1부터의 유사도 추출
        row_sim = similarity_matrix[i, i+1:]
        similar_indices = np.where(row_sim >= similarity_threshold)[0] + i + 1
        to_remove.update(similar_indices.tolist())
    
    print(f"제거할 문장 수: {len(to_remove)}")
    df = df.drop(list(to_remove)).reset_index(drop=True)
    
    # 9. 결과 CSV 저장
    print("전처리 완료. CSV 파일 저장 중...")
    df.to_csv(output_path, index=False)
    print("CSV 파일 저장 완료:", output_path)
    
    return df

if __name__ == "__main__":
    file_path = "daily_conversation_from_train.csv"
    output_path = "daily_conversation_preprocessed.csv"
    
    df_processed = preprocess_csv_with_embedding_gpu(
        file_path=file_path,
        output_path=output_path,
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # 한국어 포함 다국어 모델
        min_length=5,
        similarity_threshold=0.7
    )
    print(df_processed.head())


CSV 파일 로드 중...
텍스트 전처리 중... (숫자, 큰따옴표 제거)
전처리 후 남은 문장 수: 12452
사용 중인 디바이스: cuda
모델 로드 중...


`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


모델이 로드되었습니다. 모델 디바이스: cuda:0
임베딩 생성 중...
GPU에서 코사인 유사도 계산 중...
유사 문장 제거 중...


중복 제거 진행 중: 100%|██████████| 12452/12452 [00:00<00:00, 802931.36it/s]

제거할 문장 수: 10472
전처리 완료. CSV 파일 저장 중...
CSV 파일 저장 완료: daily_conversation_preprocessed.csv
                                                text
0  오늘은 무슨 영화를 볼까? 나는 전쟁 영화가 보고 싶어. 그런데 너는 어떤 영화를 ...
1  네가 요즘 사진 찍는 것에 관심이 많은 것 같아서 물어보려고 해. 어떤 사진을 찍고...
2  나는 요즘 새로운 언어를 배우고 싶어. 그런데 어떤 언어를 배워야 할지 결정할 수가...
3  나는 요즘 요가에 관심이 많아져서 요가 수업을 듣고 싶어. 그래? 요가는 스트레스를...
4  나는 요즘 환경에 대해 더 많은 관심을 가지고 있어. 그리고 내가 어떻게 하면 환경...
